**Installing Necessary Packages**

In [ ]:
pip install sentence-transformers==3.0.1

In [3]:
pip install numpy==1.26.4

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install wikipedia-api==0.6.0

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ollama==0.2.1

   ---------------------------------------- 0.0/75.6 kB ? eta -:--:--
   --------------------- ------------------ 41.0/75.6 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 75.6/75.6 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.26.0
    Uninstalling httpx-0.26.0:
      Successfully uninstalled httpx-0.26.0
Note: you may need to restart the kernel to use updated packages.


In [13]:
!ollama pull llama3

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ´ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ™ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest 
pulling 6a0746a1ec1a...   0% â–•                â–�    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% â–•                â–�    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% â–•                â–� 8.0 KB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% â–•                â–�  20 KB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% â–•                â–�  52 KB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% â–•                â–�  59 K

### Imports

In [17]:
import ollama
import numpy as np
import textwrap
from sentence_transformers import SentenceTransformer
from wikipediaapi import Wikipedia
import warnings
warnings.filterwarnings('ignore')




### Llama3 Function

In [3]:
def llama(query):
    response = ollama.chat(model='llama3',messages=[
    {
        'role':'user',
        'content':query,
        },
    ])
    return response['message']['content']

## Let's ask Llama about Lewis hamilton, the F1 racer

In [15]:
print(llama("Who is Lewis Hamilton?"))

Lewis Carl Davidson Hamilton is a British racing driver who has been competing in Formula One (F1) since 2007. He is widely regarded as one of the greatest drivers in F1 history.

Born on January 7, 1985, in Stevenage, Hertfordshire, England, Hamilton began karting at the age of eight and quickly rose through the ranks to become a dominant force in junior motorsports. In 2003, he became the youngest winner of the British Karting Championship, and in 2004, he won the McLaren Formula Three Cup.

Hamilton's F1 career started with McLaren in 2007, where he finished second in his debut season behind Ferrari driver Kimi Räikkönen. He then moved to Mercedes AMG Petronas in 2013, where he has remained ever since.

Throughout his career, Hamilton has achieved numerous records and milestones:

* Seven-time Formula One World Drivers' Champion (2008, 2014, 2015, 2017, 2018, 2019, and 2020)
* Most consecutive points finishes in F1 history (48)
* Fastest lap record holder with over 59 fastest laps
*

In [19]:
print(llama("When did Lewis Hamilton awarded the title 'Sir'?"))

Lewis Hamilton was appointed a Knight Bachelor by Queen Elizabeth II on December 27, 2009, and was formally addressed as "Sir" from then on. He received this honor for his services to motorsport and charity work.


## The above answer is wrong as Lewis received the title only in 2021

# Fetching Lewis Hamilton's Wikipedia Article's Text for RAG

In [21]:
wiki = Wikipedia('RAGBot/0.0','en')
doc = wiki.page('Lewis_Hamilton').text
print(doc)

Sir Lewis Carl Davidson Hamilton   (born 7 January 1985) is a British racing driver competing in Formula One, driving for Mercedes. Hamilton has won a joint-record seven Formula One World Drivers' Championship titles (tied with Michael Schumacher), and holds the records for most wins (104), pole positions (104), and podium finishes (200), among others.
Born and raised in Stevenage, Hertfordshire, Hamilton joined the McLaren Young Driver Programme in 1998. This led to a Formula One drive with McLaren from 2007 to 2012, making him the first black driver to race in the series. In his debut season, Hamilton set numerous records as he finished runner-up to Kimi Räikkönen by one point. The following season, he won his maiden title in dramatic fashion—making a crucial overtake on the last lap of the last race of the season—to become the then-youngest ever Formula One World Champion. After six years with McLaren, Hamilton signed with Mercedes in 2013.
Changes to the regulations for 2014 mandat

## Let's create chunks of data better undersatnding from the wikipedia article

In [23]:
paragraphs = doc.split('\n\n')
for i, p in enumerate(paragraphs):
    wrapped_text = textwrap.fill(p, width=100)
    print('-'*100)
    print(wrapped_text)
    print('-'*100)
    

----------------------------------------------------------------------------------------------------
Sir Lewis Carl Davidson Hamilton   (born 7 January 1985) is a British racing driver competing in
Formula One, driving for Mercedes. Hamilton has won a joint-record seven Formula One World Drivers'
Championship titles (tied with Michael Schumacher), and holds the records for most wins (104), pole
positions (104), and podium finishes (200), among others. Born and raised in Stevenage,
Hertfordshire, Hamilton joined the McLaren Young Driver Programme in 1998. This led to a Formula One
drive with McLaren from 2007 to 2012, making him the first black driver to race in the series. In
his debut season, Hamilton set numerous records as he finished runner-up to Kimi Räikkönen by one
point. The following season, he won his maiden title in dramatic fashion—making a crucial overtake
on the last lap of the last race of the season—to become the then-youngest ever Formula One World
Champion. After six 

## Loading SentenceTransformer model for embeddings

In [67]:
model = SentenceTransformer('Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True)

### Creating embeddings for the chucked paragraphs

In [69]:
embeddings = model.encode(paragraphs, normalize_embeddings=True)
embeddings.shape

(59, 768)

In [70]:
embeddings[0]

array([ 1.64436921e-02, -1.72362253e-02, -2.52244342e-02,  1.56817343e-02,
        1.36731174e-02, -1.14607168e-02,  5.17216772e-02,  2.29064119e-03,
        2.74681654e-02,  1.30020212e-02, -8.43336061e-03,  5.59847318e-02,
        2.65204590e-02, -1.47478441e-02, -6.30235812e-03, -5.40299378e-02,
        2.67903415e-05,  1.44034000e-02,  3.14046592e-02, -1.86289605e-02,
       -6.35063043e-03, -4.11796942e-02, -3.21281254e-02, -1.76908895e-02,
        2.04162691e-02,  3.97121496e-02,  6.50855750e-02,  2.77674645e-02,
        1.00459559e-02,  1.55150257e-02, -1.41718809e-03,  1.44376000e-02,
       -5.14299124e-02, -2.18789256e-03,  2.77574006e-02, -1.15572885e-02,
        4.83359955e-03,  4.98110289e-03,  9.04495828e-03,  2.51920568e-03,
       -6.57721311e-02, -6.44186065e-02, -2.62000114e-02, -5.39808422e-02,
       -5.32448255e-02, -2.33691279e-02, -2.58451160e-02, -2.46811565e-02,
        2.74329577e-02,  3.27036418e-02, -4.05275561e-02,  4.70280424e-02,
       -5.95202670e-02, -

### Create embedding of User's Query

In [73]:
query = "When was Lewis Hamilton awarded the title 'Sir'?"
embedded_query = model.encode(query, normalize_embeddings=True)

In [75]:
embedded_query.shape

(768,)

## Calculating similarities between embedded User query and chunked paragraphs

In [77]:
similarities = np.dot(embeddings,embedded_query.T)
similarities.shape

(59,)

In [79]:
similarities

array([0.69952124, 0.4772747 , 0.5062984 , 0.49489474, 0.49727875,
       0.42575198, 0.4793766 , 0.5246732 , 0.496648  , 0.44914663,
       0.49693647, 0.549172  , 0.534922  , 0.39807326, 0.4565923 ,
       0.41139153, 0.43060312, 0.44400954, 0.48455858, 0.5025195 ,
       0.5026033 , 0.515941  , 0.5804283 , 0.472422  , 0.5631921 ,
       0.41874766, 0.4218417 , 0.42751715, 0.50008464, 0.3799763 ,
       0.47615165, 0.49334574, 0.53765094, 0.45578626, 0.44945765,
       0.3960344 , 0.4669524 , 0.4422002 , 0.518166  , 0.47407365,
       0.5091237 , 0.45479515, 0.4971994 , 0.41124082, 0.5570632 ,
       0.20542586, 0.2748642 , 0.18327928, 0.27044147, 0.34420198,
       0.42500746, 0.4933339 , 0.5419401 , 0.6168394 , 0.41662925,
       0.5078142 , 0.42495313, 0.23412779, 0.348705  ], dtype=float32)

## Fetching top 3 embeddings similar to user's query

In [81]:
top_3_indexes = np.argsort(similarities, axis=0)[-3:][::-1].tolist()
top_3_indexes

[0, 53, 22]

## Using top 3 embeddings to retrieve their corresponding text paragraphs

In [83]:
similar_docs = [paragraphs[i] for i in top_3_indexes]
DOCUMENT = ""
for i, p in enumerate(similar_docs):
    wrapped_text = textwrap.fill(p, width=100)
    print('_'*100)
    print(wrapped_text)
    print('_'*100)
    DOCUMENT += wrapped_text+'\n\n'

____________________________________________________________________________________________________
Sir Lewis Carl Davidson Hamilton   (born 7 January 1985) is a British racing driver competing in
Formula One, driving for Mercedes. Hamilton has won a joint-record seven Formula One World Drivers'
Championship titles (tied with Michael Schumacher), and holds the records for most wins (104), pole
positions (104), and podium finishes (200), among others. Born and raised in Stevenage,
Hertfordshire, Hamilton joined the McLaren Young Driver Programme in 1998. This led to a Formula One
drive with McLaren from 2007 to 2012, making him the first black driver to race in the series. In
his debut season, Hamilton set numerous records as he finished runner-up to Kimi Räikkönen by one
point. The following season, he won his maiden title in dramatic fashion—making a crucial overtake
on the last lap of the last race of the season—to become the then-youngest ever Formula One World
Champion. After six 

## Integrating User query and retreived paragraphs

In [86]:
prompt = f"""
use the following DOCUMENT to answer the QUESTION.
If you don't know the answer, just say you don't know, don't try to make up an answer.

DOCUMENT: {DOCUMENT}
QUESTION: {query}
"""

# Llama+RAG function

In [89]:
response = ollama.chat(model='llama3',messages=[
        {
            'role':'user',
            'content':prompt,
            },
    ])
Llama_RAG_answer = response['message']['content']

## Llama+RAG's Answer

In [91]:
print(Llama_RAG_answer)

According to the document, Lewis Hamilton was knighted in the 2021 New Year Honours.


## Llama's Answer

In [93]:
print(llama("When did Lewis Hamilton awarded the title 'Sir'?"))

I think there may be some confusion here!

Lewis Hamilton, the British Formula One driver, was not awarded the title "Sir". However, he was knighted by Queen Elizabeth II in 2009 for his services to motorsport. This means that he is now known as Sir Lewis Carl Davidson Hamilton.

The knighthood was conferred upon him on December 31, 2009, after his first Formula One World Championship victory in 2008.
